# 1. IMPORT

In [ ]:
import json 
import pandas as pd
from collections import deque

termid = 55 # legislatura
start_date = '2015-01-02'

PROV = 'cam'
JSON_PATH =  'datasets/camara/json/'
JSON2_PATH =  'datasets/camara2/json/'

AGENTS_PATH = 'datasets/slp/agents/'
MEMBERSHIPS_PATH = 'datasets/slp/memberships/'
ORGANIZATIONS_PATH = 'datasets/slp/organizations/'
POSTS_PATH = 'datasets/slp/posts/'

FILE_SUFFIX = '55'
SENADO_URI = '81311052-e5b6-46fe-87ba-83865fa0ffb0' 
